In [2]:
# !python3 -m venv analysis-venv
!source analysis-venv/bin/activate


In [3]:
# pip install --upgrad pandas

In [4]:
!pip install numpy==1.26.4

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
import numpy as np
import joblib

os.getcwd()

'/lustre/home/rahmanr12/github-toxic/verifier-input-feature-generator/verifier-training'

In [6]:
!ls

active-learning
analysis-flan-t5-base.ipynb
analysis-llama-3.2-3B-instruct.ipynb
analysis-text-descriptive-features.ipynb
analysis-venv
capital_letters_toxicity.png
comment_distribution.png
conversationFormatter-openai-api.ipynb
dataset
feature_importance.png
original_vs_formatted_diff_generator.ipynb
output
result_merger.ipynb
select_incorrect_data-only_incorrect-v008-model_with_qwen_added_200.ipynb
select_incorrect_data-only_incorrect-v009-model_with_qwen-final.ipynb
test.ipynb
verifier_model_selected_feature_oversampling-bert.ipynb
verifier_model_selected_feature_oversampling-best.ipynb
verifier_model_selected_feature_oversampling-experiment_with_top_features.ipynb
verifier_model_selected_feature_oversampling-LOOCV.ipynb
verifier_model_selected_feature_oversampling-v1.ipynb
verifier_model_selected_feature_oversampling-v2-miller_data.ipynb
verifier_model_selected_feature_oversampling-v3-handcrafted_features.ipynb
verifier_model_selected_feature_oversampling-v4-miller_data_handcrafted

In [7]:
!cd analysis

/bin/bash: line 1: cd: analysis: No such file or directory


In [8]:
# new_directory = '/lustre/home/rahmanr12/github-toxic/verifier-input-feature-generator/analysis'

# # Change the current working directory
# os.chdir(new_directory)

In [9]:
os.getcwd()

'/lustre/home/rahmanr12/github-toxic/verifier-input-feature-generator/verifier-training'

In [10]:
# %pip uninstall scikit-learn imbalanced-learn
%pip install scikit-learn==1.3.0 imbalanced-learn==0.11.0
!pip install emoji


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


In [11]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from IPython.display import display
import re
import emoji


/lustre/home/rahmanr12/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [12]:
model_dir_name = 'flan-t5-base'

In [13]:
random_state = 42
np.random.seed(random_state)


In [ ]:
# new_labeled_df1 = pd.read_csv(file_name)
# new_labeled_df2 = pd.read_csv(file_name)
# new_labeled_df = pd.concat([new_labeled_df1, new_labeled_df2], ignore_index=True)

# new_labeled_urls = new_labeled_df['issue_url'].tolist()
# new_labeled_df

In [15]:
# new_labeled_df.iloc[0]['issue_url']

In [ ]:
# ./output/qwen2.5-32b-instruct/non_labeled_english_conversations_2024_toxicity_and_explanation_prediction_temp_0_single_prompt_part1.csv'
file_path = '/path/to/input/files' % replace with real path
df_prediction_explanation_llama = pd.read_csv(file_name)
# df_prediction_explanation_llama=df_prediction_explanation_llama.iloc[:25000]

df_prediction_explanation_llama = df_prediction_explanation_llama.rename(columns={
    'toxicity_score': 'toxicity_score_llama',
    'toxicity_explanation':'toxicity_explanation_llama'
})

file_path = '/path/to/input/files' % replace with real path
df_prediction_explanation_qwen = pd.read_csv(file_name)
# df_prediction_explanation_qwen=df_prediction_explanation_qwen.iloc[:25000]
df_prediction_explanation_qwen = df_prediction_explanation_qwen.rename(columns={
    'issue_id':'issue_url',
    'toxicity_score': 'toxicity_score_qwen',
    'toxicity_explanation':'toxicity_explanation_qwen'
})

df_prediction_explanation = pd.merge(
    df_prediction_explanation_llama[['issue_url', 'toxicity_score_llama', 'toxicity_explanation_llama']],
    df_prediction_explanation_qwen[['issue_url', 'toxicity_score_qwen', 'toxicity_explanation_qwen']],
    on='issue_url')

df_prediction_explanation['is_toxic_llm_pred_llama']= df_prediction_explanation['toxicity_score_llama']>=0.3
df_prediction_explanation['is_toxic_llm_pred_qwen']= df_prediction_explanation['toxicity_score_qwen']>=0.3

df_prediction_explanation['avg_toxicity_score'] = (df_prediction_explanation['toxicity_score_llama']+df_prediction_explanation['toxicity_score_qwen'])/2

df_prediction_explanation['toxicity_score_diff'] = abs(df_prediction_explanation['toxicity_score_llama']-df_prediction_explanation['toxicity_score_qwen'])


df_prediction_explanation['issue_url'] = df_prediction_explanation['issue_url'].astype(str)

df_prediction_explanation



,issue_url,toxicity_score_llama,toxicity_explanation_llama,toxicity_score_qwen,toxicity_explanation_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,The conversation contains entitled and unprofe...,0.1,The conversation does not contain any disrespe...,False,False,0.150,0.10
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,The conversation contains multiple instances o...,0.6,The conversation contains elements of insult a...,True,True,0.750,0.30
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,"The conversation contains some toxic elements,...",0.0,The conversation is a typical exchange of tech...,False,False,0.100,0.20
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,"The conversation is non-toxic, with a polite a...",0.0,The conversation is a technical discussion abo...,False,False,0.000,0.00
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,The conversation contains a ModuleNotFoundErro...,0.0,The conversation is a technical discussion abo...,False,False,0.025,0.05
...,...,...,...,...,...,...,...,...,...
105869,https://api.github.com/repos/ytdl-org/youtube-...,0.20,The conversation contains some technical and f...,0.0,The conversation is a collaborative and respec...,False,False,0.100,0.20
105870,https://api.github.com/repos/ytdl-org/youtube-...,0.05,The conversation contains some technical issue...,0.0,The text does not contain any toxic content. I...,False,False,0.025,0.05
105871,https://api.github.com/repos/ytdl-org/youtube-...,0.10,The conversation contains some unprofessional ...,0.0,The conversation is focused on technical issue...,False,False,0.050,0.10
105872,https://api.github.com/repos/ytdl-org/youtube-...,0.05,The conversation contains some unprofessional ...,0.0,The conversation does not contain any disrespe...,False,False,0.025,0.05


In [17]:
print(df_prediction_explanation.columns)

Index(['issue_url', 'toxicity_score_llama', 'toxicity_explanation_llama',
       'toxicity_score_qwen', 'toxicity_explanation_qwen',
       'is_toxic_llm_pred_llama', 'is_toxic_llm_pred_qwen',
       'avg_toxicity_score', 'toxicity_score_diff'],
      dtype='object')


In [ ]:

# Filter the DataFrame based on the given conditions
filtered_df = df_prediction_explanation[
    (df_prediction_explanation['toxicity_score_llama'].isnull()) |  # Check for None/NaN
    (df_prediction_explanation['toxicity_score_llama'].apply(lambda x: len(x) > 1 if isinstance(x, str) else False)) |  # Length > 1
    (df_prediction_explanation['toxicity_score_llama'] == "")|
    (df_prediction_explanation['toxicity_score_qwen'].isnull()) |  # Check for None/NaN
    (df_prediction_explanation['toxicity_score_qwen'].apply(lambda x: len(x) > 1 if isinstance(x, str) else False)) |  # Length > 1
    (df_prediction_explanation['toxicity_score_qwen'] == "")  # Empty string
]

# Extract the 'issue_url' values
invalid_issue_urls = filtered_df['issue_url']

# Display the resulting issue URLs
invalid_issue_urls.tolist()

['https://api.github.com/repos/All-Hands-AI/OpenHands/issues/2014',
 'https://api.github.com/repos/All-Hands-AI/OpenHands/issues/2059',
 'https://api.github.com/repos/All-Hands-AI/OpenHands/issues/2865',
 'https://api.github.com/repos/All-Hands-AI/OpenHands/issues/645',
 'https://api.github.com/repos/All-Hands-AI/OpenHands/issues/823',
 'https://api.github.com/repos/All-Hands-AI/OpenHands/issues/922',
 'https://api.github.com/repos/HeyPuter/puter/issues/143',
 'https://api.github.com/repos/HeyPuter/puter/issues/450',
 'https://api.github.com/repos/HeyPuter/puter/issues/85',
 'https://api.github.com/repos/Homebrew/brew/issues/16669',
 'https://api.github.com/repos/Kong/insomnia/issues/7393',
 'https://api.github.com/repos/NativeScript/NativeScript/issues/10511',
 'https://api.github.com/repos/OpenBB-finance/OpenBB/issues/6392',
 'https://api.github.com/repos/OpenBB-finance/OpenBB/issues/6703',
 'https://api.github.com/repos/Textualize/textual/issues/4867',
 'https://api.github.com/repos

In [19]:
len(invalid_issue_urls)

382

In [20]:
second_person_pronouns = ['you', 'your', 'yours', 'yourself', 'yourselves']
def count_second_person_pronouns(text):
    count = 0
    words = text.lower().split()
    for pronoun in second_person_pronouns:
        count += words.count(pronoun)
    return count

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_comment_data = pd.read_csv(file_name)

df_comment_data

,id,url,issue_url,created_at,user,author_association,body
0,2563460744,https://api.github.com/repos/photoprism/photop...,https://api.github.com/repos/photoprism/photop...,2024-10-03 08:41:09,keif888,NONE,This pull request is built on top of the great...
1,2393512575,https://api.github.com/repos/photoprism/photop...,https://api.github.com/repos/photoprism/photop...,2024-10-04T11:42:33Z,keif888,NONE,I have set this to draft whilst I investigate ...
2,2553777144,https://api.github.com/repos/photoprism/photop...,https://api.github.com/repos/photoprism/photop...,2024-09-27 21:43:24,AsikNasik,COLLABORATOR,<!--\r\n\r\nThank you for your interest in con...
3,2380111277,https://api.github.com/repos/photoprism/photop...,https://api.github.com/repos/photoprism/photop...,2024-09-27T21:43:29Z,CLAassistant,NONE,[![CLA assistant check](https://cla-assistant....
4,2550454050,https://api.github.com/repos/photoprism/photop...,https://api.github.com/repos/photoprism/photop...,2024-09-26 12:34:13,anthraxn8b,NONE,#### 1. What is not working as documented?\r\n...
...,...,...,...,...,...,...,...
454707,1987358247,https://api.github.com/repos/pbatard/rufus/iss...,https://api.github.com/repos/pbatard/rufus/iss...,2024-03-10T20:51:06Z,pbatard,OWNER,A strong hint that you haven't properly read t...
454708,1987357401,https://api.github.com/repos/pbatard/rufus/iss...,https://api.github.com/repos/pbatard/rufus/iss...,2024-03-10T20:47:40Z,Flavio93Zena,NONE,"I've read those, that's why there's the ""coinc..."
454709,1987352812,https://api.github.com/repos/pbatard/rufus/iss...,https://api.github.com/repos/pbatard/rufus/iss...,2024-03-10T20:27:55Z,pbatard,OWNER,And thank you for reading these two very relev...
454710,2104044339,https://api.github.com/repos/pbatard/rufus/iss...,https://api.github.com/repos/pbatard/rufus/iss...,2024-01-28 08:47:46,bella485,CONTRIBUTOR,"Circle Linux published 8.x and 9.x , some use..."


In [22]:
print(len(set(df_comment_data['url'])))
print(len(set(df_comment_data['issue_url'])))

454712
105879


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_conversation = pd.read_csv(file_name)
df_conversation

,issue_url,conversation,repo,id,comments_url,created_at,user,author_association,locked,comments_count,title
0,https://api.github.com/repos/2noise/ChatTTS/is...,"ibigquant: Added a Colab, BigQuant AIStudio an...",2noise/ChatTTS,2325711075,https://api.github.com/repos/2noise/ChatTTS/is...,2024-05-30 13:36:30,ibigquant,NONE,0,1,"Added a Colab, BigQuant AIStudio and Huggingfa..."
1,https://api.github.com/repos/2noise/ChatTTS/is...,dnlsndr: It is quite obvious that you've bough...,2noise/ChatTTS,2327047532,https://api.github.com/repos/2noise/ChatTTS/is...,2024-05-31 06:19:16,dnlsndr,NONE,0,7,You obviously bought github stars
2,https://api.github.com/repos/2noise/ChatTTS/is...,lyzhang1999: When I try to run this code:\r\n\...,2noise/ChatTTS,2320600870,https://api.github.com/repos/2noise/ChatTTS/is...,2024-05-28 09:41:03,lyzhang1999,NONE,0,3,WARNING:ChatTTS.core:vocos not initialized.
3,https://api.github.com/repos/2noise/ChatTTS/is...,TaurusOmar: In the forward method of the DVAE ...,2noise/ChatTTS,2328097515,https://api.github.com/repos/2noise/ChatTTS/is...,2024-05-31 15:30:17,TaurusOmar,NONE,0,1,Unnecessary memory consumption with forward me...
4,https://api.github.com/repos/2noise/ChatTTS/is...,398074715: INFO:ChatTTS.core:All initialized.\...,2noise/ChatTTS,2328288722,https://api.github.com/repos/2noise/ChatTTS/is...,2024-05-31 17:26:39,398074715,NONE,0,1,Windows环境能运行吗？requirements里没有nemo_text_process...
...,...,...,...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,tansy: <!--\r\n\r\n###########################...,ytdl-org/youtube-dl,2554848355,https://api.github.com/repos/ytdl-org/youtube-...,2024-09-29 10:02:37,tansy,NONE,0,6,dailymotion urlopen error [Errno 1] _ssl.c:504...
105875,https://api.github.com/repos/ytdl-org/youtube-...,xvtstephen: <!--\r\n\r\n######################...,ytdl-org/youtube-dl,2554986377,https://api.github.com/repos/ytdl-org/youtube-...,2024-09-29 15:21:19,xvtstephen,NONE,0,2,"Unable to extract nsig jsi,"
105876,https://api.github.com/repos/ytdl-org/youtube-...,draconis10: - [ x] I'm reporting a broken site...,ytdl-org/youtube-dl,2557871823,https://api.github.com/repos/ytdl-org/youtube-...,2024-09-30 23:48:07,draconis10,NONE,0,5,Pornhub artists from shared site
105877,https://api.github.com/repos/ytdl-org/youtube-...,GregoriusT: <!--\r\n\r\n######################...,ytdl-org/youtube-dl,2557990318,https://api.github.com/repos/ytdl-org/youtube-...,2024-10-01 01:46:58,GregoriusT,NONE,0,1,Youtube Shorts Playlist on Channels will resul...


In [24]:
df_comment_data['body'] = df_comment_data['body'].astype(str).replace({'nan': '', '': ''})
df_comment_data['body'] = df_comment_data['body'].astype(str).fillna('')
df_comment_data['pronoun_count'] = df_comment_data['body'].apply(count_second_person_pronouns)
df_comment_data['comment_length'] = df_comment_data['body'].apply(len)


In [25]:

# First, group by 'issue_url' and 'speaker_name' and count the comments per speaker
speaker_comment_counts = df_comment_data.groupby(['issue_url', 'user']).size()
speaker_comment_counts

issue_url                                                      user       
https://api.github.com/repos/2noise/ChatTTS/issues/104         fumiama        1
                                                               ibigquant      1
https://api.github.com/repos/2noise/ChatTTS/issues/129         Richard0403    1
                                                               dnlsndr        2
                                                               freeok         1
                                                                             ..
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32936  draconis10     3
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32937  GregoriusT     1
                                                               dirkf          1
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32938  daixtrose      1
                                                               dirkf          1
Length: 259515, dtype: int64

In [26]:
max_comments_per_issue_by_speakers = speaker_comment_counts.groupby(level=0).max()
max_comments_per_issue_by_speakers

issue_url
https://api.github.com/repos/2noise/ChatTTS/issues/104           1
https://api.github.com/repos/2noise/ChatTTS/issues/129           2
https://api.github.com/repos/2noise/ChatTTS/issues/14            1
https://api.github.com/repos/2noise/ChatTTS/issues/148           1
https://api.github.com/repos/2noise/ChatTTS/issues/150           1
                                                                ..
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32934    4
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32935    3
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32936    3
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32937    1
https://api.github.com/repos/ytdl-org/youtube-dl/issues/32938    1
Length: 105879, dtype: int64

In [27]:
# Define a function to count '@' in each comment
def count_at_signs(text):
    text = text or ''
    
    try:
        return text.count('@')
    except Exception as e:
        # Catch all other exceptions
        # print("An error occurred:", e)
        return 0
    
def count_quotes(text):
    try:
        return text.count("'") + text.count('"') + text.count('`')
    except Exception as e:
        # print("An error occurred:", e)
        return 0
    
def count_hashes(text):
    try:
        return text.count("#")
    except Exception as e:
        # print("An error occurred:", e)
        return 0

In [28]:
def count_words(text):
    return len(str(text).split())


# Get max number of comments per speaker per issue
median_comments_per_issue_by_speakers = speaker_comment_counts.groupby(level=0).median()

# Group by 'issue_url' and aggregate
aggregated_data = df_comment_data.groupby('issue_url').agg(
    comment_counts=('url', 'count'),  # Count of comments
    unique_speakers=('user', 'nunique'),  # Count of unique speaker names
    total_second_person_pronouns=('pronoun_count', 'sum'),
    max_comment_length=('comment_length', 'max'),  # Maximum length of comments,
    total_ats=('body', lambda x: x.apply(count_at_signs).sum()),  # Sum of '@' in all comments
    total_quotes=('body', lambda x: x.apply(count_quotes).sum()),  # Sum of quote characters in all comments
    median_words_in_comment=('body', lambda x: x.apply(count_words).median()),
    std_dev_words_in_comment=('body', lambda x: x.apply(count_words).std()),
    max_words_in_comment=('body', lambda x: x.apply(count_words).max())
)

aggregated_data['max_comments_by_one_speaker'] = aggregated_data.index.map(max_comments_per_issue_by_speakers)
aggregated_data['median_comments_by_one_speaker'] = aggregated_data.index.map(median_comments_per_issue_by_speakers)


# Reset index to make 'issue_url' a column again
aggregated_data.reset_index(inplace=True)
# aggregated_data=aggregated_data[:25000]

# Print the result
aggregated_data


,issue_url,comment_counts,unique_speakers,total_second_person_pronouns,max_comment_length,total_ats,total_quotes,median_words_in_comment,std_dev_words_in_comment,max_words_in_comment,max_comments_by_one_speaker,median_comments_by_one_speaker
0,https://api.github.com/repos/2noise/ChatTTS/is...,2,2,1,149,0,1,20.0,14.142136,30,1,1.0
1,https://api.github.com/repos/2noise/ChatTTS/is...,8,6,15,626,3,7,22.0,37.159646,107,2,1.0
2,https://api.github.com/repos/2noise/ChatTTS/is...,4,4,2,409,1,32,16.0,14.545904,42,1,1.0
3,https://api.github.com/repos/2noise/ChatTTS/is...,2,2,1,719,0,12,54.5,57.275649,95,1,1.0
4,https://api.github.com/repos/2noise/ChatTTS/is...,2,2,2,1506,0,8,72.0,96.166522,140,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,7,2,10,4755,0,112,150.0,196.462040,581,4,3.5
105875,https://api.github.com/repos/ytdl-org/youtube-...,3,1,12,9213,0,72,9.0,561.184462,981,3,3.0
105876,https://api.github.com/repos/ytdl-org/youtube-...,6,2,2,3479,0,167,76.0,131.067540,373,3,3.0
105877,https://api.github.com/repos/ytdl-org/youtube-...,2,2,6,3108,2,23,238.5,280.721392,437,1,1.0


In [ ]:
def check_is_closed(group):
    last_two_comments = group.tail(2)  # Get the last 2 comments for the issue_url
    return int(last_two_comments['body'].str.contains(r'\bclose\b', case=False).any())

# Group by issue_url and apply the function
aggregated_data2 = df_comment_data.groupby('issue_url').apply(
    lambda group: pd.Series({'is_issue_closed': check_is_closed(group)})
).reset_index()


def count_emojis(text):
    """
    Count the total number of emojis in a text string.
    
    Args:
        text (str): The text string to analyze (e.g., GitHub issue comment)
    
    Returns:
        int: Total number of emojis found
    """
    # Get all emojis present in the text
    emoji_list = [c for c in text if c in emoji.EMOJI_DATA]
    
    # Return the count
    return len(emoji_list)

def calculate_emoji_metrics(group):
    # Calculate total emojis across all comments for an issue_url
    total_emojis = group['body'].apply(count_emojis).sum()
    
    # Count the number of comments containing at least one emoji
    total_comments_with_emoji = group['body'].apply(lambda x: count_emojis(x) > 0).sum()
    total_comments = len(group)

    return pd.Series({
        'emoji_count': total_emojis,
        'total_comment_with_emoji': total_comments_with_emoji,
        'total_comment_with_emoji_ratio': total_comments_with_emoji/total_comments
    })

# Group by issue_url and apply the function
emoji_metrics = df_comment_data.groupby('issue_url').apply(calculate_emoji_metrics).reset_index()

# Merge with the aggregated_data2
aggregated_data2 = pd.merge(aggregated_data2, emoji_metrics, on='issue_url')

# aggregated_data2['code_of_conduct_mentioned'] = df_comment_data.groupby('issue_url')['text'].apply(
#     lambda comments: int(comments.str.contains("code of conduct", case=False).any())
# ).reset_index(drop=True)

def get_first_coc_mention_idx(comments):
    # Find the first occurrence of "code of conduct" in comments
    mask = comments.str.contains("code of conduct", case=False)
    # If found, return the index (starting from 1), else return 0
    if mask.any():
        return mask.idxmax() + 1  # Adding 1 to convert from 0-based to 1-based indexing
    return 0

# Apply the function to get the index of first code of conduct mention
aggregated_data2['code_of_conduct_mentioned_comment_idx'] = df_comment_data.groupby('issue_url')['body'].apply(
    get_first_coc_mention_idx
).reset_index(drop=True)

aggregated_data2

/tmp/ipykernel_186996/2154823758.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_data2 = df_comment_data.groupby('issue_url').apply(
/tmp/ipykernel_186996/2154823758.py:125: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  emoji_metrics = df_comment_data.groupby('issue_url').apply(calculate_emoji_metrics).reset_index()


,issue_url,is_issue_closed,emoji_count,total_comment_with_emoji,total_comment_with_emoji_ratio,code_of_conduct_mentioned_comment_idx
0,https://api.github.com/repos/2noise/ChatTTS/is...,1,0.0,0.0,0.0,0
1,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0
2,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0
3,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0
4,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,1,0.0,0.0,0.0,0
105875,https://api.github.com/repos/ytdl-org/youtube-...,0,0.0,0.0,0.0,0
105876,https://api.github.com/repos/ytdl-org/youtube-...,0,0.0,0.0,0.0,0
105877,https://api.github.com/repos/ytdl-org/youtube-...,0,0.0,0.0,0.0,0


In [30]:
# Technical features
def has_stack_trace(text):
    st_regex = re.compile(r'at [a-zA-Z0-9\.<>$]+\(.+\)')
    return bool(st_regex.search(text))

def check_template(first_comment):
    template_indicators = ['### Description', '## Description', '### Steps to reproduce', '## Steps to reproduce']
    return int(any(indicator in first_comment for indicator in template_indicators))

# Update aggregation
def calculate_technical_features(group):
    texts = group['body'].astype(str)
    first_comment = texts.iloc[0] if not texts.empty else ""
    
    return pd.Series({
        'has_stack_trace': int(texts.apply(has_stack_trace).any()),
        'has_code': int(texts.str.contains(r'```\w*\n.*?\n```', flags=re.DOTALL).any()),
        # 'has_markdown': int(texts.str.contains(r'```').any()),
        'has_template': check_template(first_comment)
    })

technical_feature_metrics = df_comment_data.groupby('issue_url').apply(calculate_technical_features).reset_index()
aggregated_data2 = pd.merge(aggregated_data2, technical_feature_metrics, on='issue_url')

/tmp/ipykernel_186996/510057303.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  technical_feature_metrics = df_comment_data.groupby('issue_url').apply(calculate_technical_features).reset_index()


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_time_features = pd.read_csv(file_name)

# df_time_features=df_time_features[:25000]
df_time_features

,Unnamed: 0,issue_url,avg_comment_time,max_comment_time,std_comment_time
0,0,https://api.github.com/repos/2noise/ChatTTS/is...,236.017639,472.035278,333.779346
1,1,https://api.github.com/repos/2noise/ChatTTS/is...,0.024028,12.434722,6.563281
2,2,https://api.github.com/repos/2noise/ChatTTS/is...,4.150903,407.386111,321.167900
3,3,https://api.github.com/repos/2noise/ChatTTS/is...,0.498333,0.996667,0.704750
4,4,https://api.github.com/repos/2noise/ChatTTS/is...,0.151111,0.302222,0.213703
...,...,...,...,...,...
105874,105874,https://api.github.com/repos/ytdl-org/youtube-...,10.385079,25.016667,9.655842
105875,105875,https://api.github.com/repos/ytdl-org/youtube-...,0.074815,0.214444,0.121026
105876,105876,https://api.github.com/repos/ytdl-org/youtube-...,5.813935,17.095556,6.318356
105877,105877,https://api.github.com/repos/ytdl-org/youtube-...,2.991389,5.982778,4.230463


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_text_descriptive = pd.read_csv(file_name)

# df_text_descriptive = df_text_descriptive.drop(columns=['is_toxic','body'])
df_text_descriptive = df_text_descriptive.rename(columns={col:  col+'_TD' if col != 'issue_url' else col for col in df_text_descriptive.columns})

# df_text_descriptive=df_text_descriptive[:25000]
df_text_descriptive

,issue_url,pos_prop_ADJ_TD,pos_prop_ADP_TD,pos_prop_ADV_TD,pos_prop_AUX_TD,pos_prop_CCONJ_TD,pos_prop_DET_TD,pos_prop_INTJ_TD,pos_prop_NOUN_TD,pos_prop_NUM_TD,...,duplicate_ngram_chr_fraction_9_TD,duplicate_ngram_chr_fraction_10_TD,top_ngram_chr_fraction_2_TD,top_ngram_chr_fraction_3_TD,top_ngram_chr_fraction_4_TD,oov_ratio_TD,entropy_TD,perplexity_TD,per_word_perplexity_TD,text_TD
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.040000,0.040000,0.040000,0.040000,0.040000,0.120000,0.000000,0.160000,0.020000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.478754,4.387477e+00,8.774953e-02,"ibigquant: Added a Colab, BigQuant AIStudio an..."
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.051813,0.067358,0.059585,0.062176,0.025907,0.069948,0.002591,0.186528,0.010363,...,0.472785,0.472785,0.018677,0.000000,0.000000,NaN,10.773878,4.775685e+04,1.237224e+02,dnlsndr: It is quite obvious that you've bough...
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.021277,0.010638,0.010638,0.010638,0.000000,0.026596,0.000000,0.117021,0.000000,...,0.060890,0.000000,0.028103,0.021077,0.029274,NaN,2.551462,1.282585e+01,6.822258e-02,lyzhang1999: When I try to run this code:\r\n\...
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.030303,0.048485,0.048485,0.054545,0.000000,0.042424,0.000000,0.121212,0.000000,...,0.360049,0.360049,0.019729,0.093711,0.103576,NaN,3.570924,3.554942e+01,2.154510e-01,TaurusOmar: In the forward method of the DVAE ...
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.029412,0.054622,0.033613,0.016807,0.008403,0.012605,0.000000,0.117647,0.079832,...,0.000000,0.000000,0.015316,0.000000,0.000000,NaN,4.285148,7.261331e+01,3.050980e-01,398074715: INFO:ChatTTS.core:All initialized.\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.042815,0.042815,0.019076,0.029250,0.013565,0.032641,0.001272,0.145825,0.024163,...,0.298814,0.284447,0.027191,0.039340,0.051683,NaN,44.706513,2.604904e+19,1.104241e+16,tansy: <!--\r\n\r\n###########################...
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.045942,0.036243,0.010720,0.020419,0.011741,0.025523,0.004084,0.169474,0.023481,...,0.530772,0.518998,0.030183,0.043669,0.057369,NaN,33.705630,4.346795e+14,2.218885e+11,xvtstephen: <!--\r\n\r\n######################...
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.028440,0.039201,0.011530,0.023828,0.008455,0.023828,0.000769,0.166795,0.017679,...,0.143874,0.143874,0.009087,0.036347,0.014993,NaN,22.981227,9.563572e+09,7.350939e+06,draconis10: - [ x] I'm reporting a broken site...
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.037080,0.041715,0.020857,0.032445,0.017381,0.037080,0.001159,0.146002,0.008111,...,0.104858,0.104858,0.083531,0.120853,0.158768,NaN,14.607064,2.206821e+06,2.557151e+03,GregoriusT: <!--\r\n\r\n######################...


In [34]:
x= df_text_descriptive['symbol_to_word_ratio_#_TD'].tolist()
x.sort()
x[-5:]

[2.137150466045273,
 2.249647390691114,
 2.659388646288209,
 2.947811447811448,
 3.242914979757085]

In [35]:
print(df_text_descriptive['symbol_to_word_ratio_#_TD'].tolist().sort())

None


In [36]:
df_text_descriptive.loc[df_text_descriptive['symbol_to_word_ratio_#_TD'] < 0.0001, 'symbol_to_word_ratio_#_TD'] = 0.0001


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_text_content_features= pd.read_csv(file_name)

df_text_content_features=df_text_content_features.drop(columns=['conversation'])

# df_text_content_features=df_text_content_features[:25000]
df_text_content_features

,issue_url,question_mark_count,negative_word_count
0,https://api.github.com/repos/2noise/ChatTTS/is...,0,0
1,https://api.github.com/repos/2noise/ChatTTS/is...,2,7
2,https://api.github.com/repos/2noise/ChatTTS/is...,2,5
3,https://api.github.com/repos/2noise/ChatTTS/is...,0,7
4,https://api.github.com/repos/2noise/ChatTTS/is...,0,3
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,6,6
105875,https://api.github.com/repos/ytdl-org/youtube-...,3,4
105876,https://api.github.com/repos/ytdl-org/youtube-...,9,8
105877,https://api.github.com/repos/ytdl-org/youtube-...,1,5


In [ ]:
emotions= [
            # 'admiration', 'amusement',
    'anger', 'annoyance', 'approval',
    # 'caring',
            'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
            'disgust', 'embarrassment', 
    # 'excitement', 'fear', 'gratitude', 'grief',
            # 'joy', 'love', 'nervousness', 'neutral', 'optimism', 
    'pride',
    # 'realization', 'relief', 'remorse', 
    'sadness', 'surprise'
        ]

file_path = '/path/to/input/files' % replace with real path
df_emotion_scores = pd.read_csv(file_path)

df_emotion_scores= df_emotion_scores[emotions+['issue_url']]
# df_emotion_scores=df_emotion_scores[:25000]
df_emotion_scores

,anger,annoyance,approval,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,pride,sadness,surprise,issue_url
0,0.000517,0.010255,0.001646,0.000303,0.000390,0.000321,0.001430,0.000647,0.000285,0.000074,0.002294,0.000082,0.997553,https://api.github.com/repos/2noise/ChatTTS/is...
1,0.002298,0.818097,0.001450,0.024719,0.005796,0.000234,0.002762,0.008083,0.000859,0.000869,0.034738,0.004598,0.876010,https://api.github.com/repos/2noise/ChatTTS/is...
2,0.001151,0.000886,0.000595,0.893923,0.664474,0.000207,0.002153,0.000588,0.000575,0.004614,0.017837,0.377676,0.998627,https://api.github.com/repos/2noise/ChatTTS/is...
3,0.000212,0.002155,0.001582,0.001132,0.000918,0.000195,0.000668,0.000693,0.000303,0.000220,0.004300,0.000400,0.998807,https://api.github.com/repos/2noise/ChatTTS/is...
4,0.000263,0.000830,0.000807,0.000708,0.001124,0.000353,0.000312,0.000488,0.000269,0.000174,0.000596,0.000212,0.999005,https://api.github.com/repos/2noise/ChatTTS/is...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105874,0.001124,0.004767,0.006616,0.367953,0.482673,0.001341,0.003856,0.001466,0.002572,0.022517,0.379808,0.002792,0.980906,https://api.github.com/repos/ytdl-org/youtube-...
105875,0.000477,0.001252,0.001579,0.001089,0.002043,0.000485,0.000226,0.000373,0.000432,0.000143,0.000635,0.000224,0.999060,https://api.github.com/repos/ytdl-org/youtube-...
105876,0.000366,0.000895,0.000478,0.000753,0.000891,0.000167,0.000242,0.000398,0.000405,0.000175,0.000343,0.000176,0.998892,https://api.github.com/repos/ytdl-org/youtube-...
105877,0.000605,0.001397,0.000423,0.024504,0.006564,0.000188,0.000369,0.001081,0.000479,0.000500,0.000985,0.000451,0.998845,https://api.github.com/repos/ytdl-org/youtube-...


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_sentiment_polarity_senticr = pd.read_csv(file_path)
df_sentiment_polarity_senticr

,issue_url,has_neg_comment_sentcr,non_neg_comment_ratio_sentcr,neg_comment_ratio_sentcr,sentiment_transition_ratio_sentcr,total_non_neg_comment,total_neg_comment,total_transitions
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,2.0,0.0,0.0
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,8.0,0.0,0.0
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,4.0,0.0,0.0
3,https://api.github.com/repos/2noise/ChatTTS/is...,1.0,0.500000,0.500000,0.500000,1.0,1.0,1.0
4,https://api.github.com/repos/2noise/ChatTTS/is...,1.0,0.500000,0.500000,0.500000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.571429,0.428571,0.714286,4.0,3.0,5.0
105875,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.666667,0.333333,0.333333,2.0,1.0,1.0
105876,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.833333,0.166667,0.333333,5.0,1.0,2.0
105877,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.500000,0.500000,0.500000,1.0,1.0,1.0


In [43]:
df_sentiment_polarity_senticr=df_sentiment_polarity_senticr[['issue_url', 'has_neg_comment_sentcr', 'non_neg_comment_ratio_sentcr',
       'neg_comment_ratio_sentcr', 'sentiment_transition_ratio_sentcr']]


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_sentiment_polarity_textblob = pd.read_csv(file_path)

# df_sentiment_polarity_textblob=df_sentiment_polarity_textblob[:25000]
df_sentiment_polarity_textblob

,issue_url,has_neg_comment_textblob,non_neg_comment_ratio_textblob,neg_comment_ratio_textblob,sentiment_transition_ratio_textblob,total_non_neg_comment_textblob,total_neg_comment_textblob,total_transitions_textblob,max_consecutive_negatives_textblob
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,2.0,0.0,0.0,0.0
1,https://api.github.com/repos/2noise/ChatTTS/is...,1.0,0.500000,0.500000,0.375000,4.0,4.0,3.0,3.0
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,4.0,0.0,0.0,0.0
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,2.0,0.0,0.0,0.0
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.0,1.000000,0.000000,0.000000,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.571429,0.428571,0.428571,4.0,3.0,3.0,2.0
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.0,1.000000,0.000000,0.000000,3.0,0.0,0.0,0.0
105876,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.666667,0.333333,0.166667,4.0,2.0,1.0,2.0
105877,https://api.github.com/repos/ytdl-org/youtube-...,1.0,0.500000,0.500000,0.500000,1.0,1.0,1.0,0.0


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_sentiment_polarity_vader = pd.read_csv(file_path)

# df_sentiment_polarity_vader=df_sentiment_polarity_vader[:25000]
df_sentiment_polarity_vader

,issue_url,has_neg_comment_vader,non_neg_comment_ratio_vader,neg_comment_ratio_vader,sentiment_transition_ratio_vader,total_non_neg_comment_vader,total_neg_comment_vader,total_transitions_vader,max_consecutive_negatives_vader
0,https://api.github.com/repos/2noise/ChatTTS/is...,False,1.000000,0.000000,0.000000,2,0,0,0
1,https://api.github.com/repos/2noise/ChatTTS/is...,True,0.250000,0.750000,0.250000,2,6,2,5
2,https://api.github.com/repos/2noise/ChatTTS/is...,True,0.500000,0.500000,0.500000,2,2,2,0
3,https://api.github.com/repos/2noise/ChatTTS/is...,False,1.000000,0.000000,0.000000,2,0,0,0
4,https://api.github.com/repos/2noise/ChatTTS/is...,False,1.000000,0.000000,0.000000,2,0,0,0
...,...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,True,0.714286,0.285714,0.428571,5,2,3,0
105875,https://api.github.com/repos/ytdl-org/youtube-...,True,0.666667,0.333333,0.666667,2,1,2,0
105876,https://api.github.com/repos/ytdl-org/youtube-...,True,0.500000,0.500000,0.500000,3,3,3,2
105877,https://api.github.com/repos/ytdl-org/youtube-...,True,0.500000,0.500000,0.500000,1,1,1,0


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_features_given_conv_exp = pd.read_csv(file_path)

# df_explanation_features_given_conv_exp=df_explanation_features_given_conv_exp[:25000]
df_explanation_features_given_conv_exp

,issue_url,toxic_label_probability,non_toxic_label_probability
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.651795,0.337840
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.825229,0.168899
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.783657,0.209603
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.673987,0.319184
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.773767,0.219132
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.713442,0.277488
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.785814,0.205298
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.728549,0.263569
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.724942,0.267114


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_features_given_conv = pd.read_csv(file_path)

# df_explanation_features_given_conv=df_explanation_features_given_conv[:25000]
df_explanation_features_given_conv

,issue_url,toxic_label_probability,non_toxic_label_probability
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.483464,0.506205
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.574902,0.418462
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.655699,0.333366
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.679470,0.314127
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.646438,0.345838
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.706250,0.284940
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.683447,0.305464
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.638400,0.351973
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.654645,0.332183


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_features_given_exp = pd.read_csv(file_path)

# df_explanation_features_given_exp=df_explanation_features_given_exp[:25000]
df_explanation_features_given_exp

,issue_url,toxic_label_probability,non_toxic_label_probability
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.802078,0.188866
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.860117,0.132877
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.796883,0.194509
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.572832,0.417573
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.771985,0.218462
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.842088,0.148857
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.817401,0.173213
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.827594,0.164082
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.823427,0.168239


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_logits_features = pd.read_csv(file_path)

df_explanation_logits_features

,issue_url,first_token_logit,first_token_entropy,avg_entropy,avg_logits
0,https://api.github.com/repos/2noise/ChatTTS/is...,1.353376,7.875906,0.377815,1.511401
1,https://api.github.com/repos/2noise/ChatTTS/is...,1.777933,6.673337,0.309320,0.820344
2,https://api.github.com/repos/2noise/ChatTTS/is...,2.790311,5.229239,0.432898,0.685354
3,https://api.github.com/repos/2noise/ChatTTS/is...,1.836710,5.731345,0.223707,0.979373
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.491421,6.739817,0.870594,0.831730
...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,2.751625,5.045059,0.256689,0.695529
105875,https://api.github.com/repos/ytdl-org/youtube-...,1.731196,6.048190,0.474795,0.790268
105876,https://api.github.com/repos/ytdl-org/youtube-...,1.556329,7.115627,4.749567,1.439919
105877,https://api.github.com/repos/ytdl-org/youtube-...,1.948191,6.621759,0.430005,1.246732


In [54]:

df_explanation_features_given_conv_exp = df_explanation_features_given_conv_exp.rename(columns=
                                                 {
                                                     'toxic_label_probability': 'toxic_prob_conv_exp',
                                                     'non_toxic_label_probability': 'non_toxic_prob_conv_exp',
                                                 })
df_explanation_features_given_conv_exp

,issue_url,toxic_prob_conv_exp,non_toxic_prob_conv_exp
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.651795,0.337840
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.825229,0.168899
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.783657,0.209603
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.673987,0.319184
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.773767,0.219132
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.713442,0.277488
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.785814,0.205298
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.728549,0.263569
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.724942,0.267114


In [55]:

df_explanation_features_given_conv = df_explanation_features_given_conv.rename(columns=
                                     {
                                         'toxic_label_probability': 'toxic_prob_conv',
                                         'non_toxic_label_probability': 'non_toxic_prob_conv',
                                     })
df_explanation_features_given_conv

,issue_url,toxic_prob_conv,non_toxic_prob_conv
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.483464,0.506205
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.574902,0.418462
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.655699,0.333366
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.679470,0.314127
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.646438,0.345838
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.706250,0.284940
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.683447,0.305464
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.638400,0.351973
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.654645,0.332183


In [56]:

df_explanation_features_given_exp = df_explanation_features_given_exp.rename(columns=
                                     {
                                         'toxic_label_probability': 'toxic_prob_exp',
                                         'non_toxic_label_probability': 'non_toxic_prob_exp',
                                     })
df_explanation_features_given_exp

,issue_url,toxic_prob_exp,non_toxic_prob_exp
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.802078,0.188866
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.860117,0.132877
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.796883,0.194509
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.572832,0.417573
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.771985,0.218462
...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.842088,0.148857
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.817401,0.173213
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.827594,0.164082
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.823427,0.168239


In [57]:
df_explanation_logits_features = df_explanation_logits_features.rename(columns=
                                     {
                                         'first_token_logit': 'exp_first_token_logit',
                                         'first_token_entropy': 'exp_first_token_entropy',
                                         'avg_entropy': 'exp_avg_entropy',
                                         'avg_logits': 'exp_avg_logits'
                                     })
df_explanation_logits_features

,issue_url,exp_first_token_logit,exp_first_token_entropy,exp_avg_entropy,exp_avg_logits
0,https://api.github.com/repos/2noise/ChatTTS/is...,1.353376,7.875906,0.377815,1.511401
1,https://api.github.com/repos/2noise/ChatTTS/is...,1.777933,6.673337,0.309320,0.820344
2,https://api.github.com/repos/2noise/ChatTTS/is...,2.790311,5.229239,0.432898,0.685354
3,https://api.github.com/repos/2noise/ChatTTS/is...,1.836710,5.731345,0.223707,0.979373
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.491421,6.739817,0.870594,0.831730
...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,2.751625,5.045059,0.256689,0.695529
105875,https://api.github.com/repos/ytdl-org/youtube-...,1.731196,6.048190,0.474795,0.790268
105876,https://api.github.com/repos/ytdl-org/youtube-...,1.556329,7.115627,4.749567,1.439919
105877,https://api.github.com/repos/ytdl-org/youtube-...,1.948191,6.621759,0.430005,1.246732


In [58]:
df_explanation_features = pd.DataFrame()
df_explanation_features = pd.merge(df_explanation_features_given_conv_exp, df_explanation_features_given_conv, on='issue_url')
df_explanation_features = pd.merge(df_explanation_features, df_explanation_features_given_exp, on='issue_url')
df_explanation_features = pd.merge(df_explanation_features, df_explanation_logits_features, on='issue_url')
df_explanation_features=df_explanation_features.drop('Unnamed: 0', axis=1, errors='ignore')
df_explanation_features



,issue_url,toxic_prob_conv_exp,non_toxic_prob_conv_exp,toxic_prob_conv,non_toxic_prob_conv,toxic_prob_exp,non_toxic_prob_exp,exp_first_token_logit,exp_first_token_entropy,exp_avg_entropy,exp_avg_logits
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.651795,0.337840,0.483464,0.506205,0.802078,0.188866,1.353376,7.875906,0.377815,1.511401
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.825229,0.168899,0.574902,0.418462,0.860117,0.132877,1.777933,6.673337,0.309320,0.820344
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.783657,0.209603,0.655699,0.333366,0.796883,0.194509,2.790311,5.229239,0.432898,0.685354
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.673987,0.319184,0.679470,0.314127,0.572832,0.417573,1.836710,5.731345,0.223707,0.979373
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.773767,0.219132,0.646438,0.345838,0.771985,0.218462,0.491421,6.739817,0.870594,0.831730
...,...,...,...,...,...,...,...,...,...,...,...
105874,https://api.github.com/repos/ytdl-org/youtube-...,0.713442,0.277488,0.706250,0.284940,0.842088,0.148857,2.751625,5.045059,0.256689,0.695529
105875,https://api.github.com/repos/ytdl-org/youtube-...,0.785814,0.205298,0.683447,0.305464,0.817401,0.173213,1.731196,6.048190,0.474795,0.790268
105876,https://api.github.com/repos/ytdl-org/youtube-...,0.728549,0.263569,0.638400,0.351973,0.827594,0.164082,1.556329,7.115627,4.749567,1.439919
105877,https://api.github.com/repos/ytdl-org/youtube-...,0.724942,0.267114,0.654645,0.332183,0.823427,0.168239,1.948191,6.621759,0.430005,1.246732


In [59]:
df_explanation_features.columns

Index(['issue_url', 'toxic_prob_conv_exp', 'non_toxic_prob_conv_exp',
       'toxic_prob_conv', 'non_toxic_prob_conv', 'toxic_prob_exp',
       'non_toxic_prob_exp', 'exp_first_token_logit',
       'exp_first_token_entropy', 'exp_avg_entropy', 'exp_avg_logits'],
      dtype='object')

In [60]:
##########################################
df_merged = aggregated_data2.copy()

df_merged = pd.merge(df_merged, df_prediction_explanation[[
    'issue_url', 'toxicity_score_llama', 'is_toxic_llm_pred_llama',
    'toxicity_score_qwen', 'is_toxic_llm_pred_qwen',
    'avg_toxicity_score', 'toxicity_score_diff'
]], on='issue_url')

# df_merged = pd.merge(df_merged, df_bert_score, on='issue_url')
df_merged = pd.merge(df_merged, df_time_features, on='issue_url')

# df_merged = pd.merge(df_merged, df_perplexity, on='issue_url')
df_merged = pd.merge(df_merged, aggregated_data, on='issue_url')

# df_merged = pd.merge(df_merged, df_tone_features, on='issue_url')

# # we may not use emotion scores
df_merged = pd.merge(df_merged, df_emotion_scores, on='issue_url')

df_merged = pd.merge(df_merged, df_sentiment_polarity_senticr, on='issue_url')
df_merged = pd.merge(df_merged, df_sentiment_polarity_textblob, on='issue_url')
df_merged = pd.merge(df_merged, df_sentiment_polarity_vader, on='issue_url')

df_merged = pd.merge(df_merged, df_text_descriptive, on='issue_url')
df_merged = pd.merge(df_merged, df_text_content_features, on='issue_url')

# df_merged = pd.merge(df_merged, df_label_logits_features, on=['issue_url'])

# df_merged = pd.merge(df_merged, df_outlier_score, on='issue_url')

# # df_merged = pd.merge(df_merged, df_explanation_features[['is_toxic', 'issue_url']], on=['issue_url'])
df_merged = pd.merge(df_merged, df_explanation_features, on=['issue_url'])
# df_merged["is_toxic"] = df_explanation_features['is_toxic']


# df_merged
df_merged = df_merged.reset_index(drop=True)

df_merged



,issue_url,is_issue_closed,emoji_count,total_comment_with_emoji,total_comment_with_emoji_ratio,code_of_conduct_mentioned_comment_idx,has_stack_trace,has_code,has_template,toxicity_score_llama,...,toxic_prob_conv_exp,non_toxic_prob_conv_exp,toxic_prob_conv,non_toxic_prob_conv,toxic_prob_exp,non_toxic_prob_exp,exp_first_token_logit,exp_first_token_entropy,exp_avg_entropy,exp_avg_logits
0,https://api.github.com/repos/2noise/ChatTTS/is...,1,0.0,0.0,0.0,0,0,0,0,0.20,...,0.651795,0.337840,0.483464,0.506205,0.802078,0.188866,1.353376,7.875906,0.377815,1.511401
1,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0,0,0,0,0.90,...,0.825229,0.168899,0.574902,0.418462,0.860117,0.132877,1.777933,6.673337,0.309320,0.820344
2,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0,0,0,0,0.20,...,0.783657,0.209603,0.655699,0.333366,0.796883,0.194509,2.790311,5.229239,0.432898,0.685354
3,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0,0,0,0,0.00,...,0.673987,0.319184,0.679470,0.314127,0.572832,0.417573,1.836710,5.731345,0.223707,0.979373
4,https://api.github.com/repos/2noise/ChatTTS/is...,0,0.0,0.0,0.0,0,0,0,0,0.05,...,0.773767,0.219132,0.646438,0.345838,0.771985,0.218462,0.491421,6.739817,0.870594,0.831730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105869,https://api.github.com/repos/ytdl-org/youtube-...,1,0.0,0.0,0.0,0,0,0,1,0.20,...,0.713442,0.277488,0.706250,0.284940,0.842088,0.148857,2.751625,5.045059,0.256689,0.695529
105870,https://api.github.com/repos/ytdl-org/youtube-...,0,0.0,0.0,0.0,0,0,0,1,0.05,...,0.785814,0.205298,0.683447,0.305464,0.817401,0.173213,1.731196,6.048190,0.474795,0.790268
105871,https://api.github.com/repos/ytdl-org/youtube-...,0,0.0,0.0,0.0,0,0,0,1,0.10,...,0.728549,0.263569,0.638400,0.351973,0.827594,0.164082,1.556329,7.115627,4.749567,1.439919
105872,https://api.github.com/repos/ytdl-org/youtube-...,0,0.0,0.0,0.0,0,0,0,1,0.05,...,0.724942,0.267114,0.654645,0.332183,0.823427,0.168239,1.948191,6.621759,0.430005,1.246732


In [61]:
df_merged.columns.tolist()

['issue_url',
 'is_issue_closed',
 'emoji_count',
 'total_comment_with_emoji',
 'total_comment_with_emoji_ratio',
 'code_of_conduct_mentioned_comment_idx',
 'has_stack_trace',
 'has_code',
 'has_template',
 'toxicity_score_llama',
 'is_toxic_llm_pred_llama',
 'toxicity_score_qwen',
 'is_toxic_llm_pred_qwen',
 'avg_toxicity_score',
 'toxicity_score_diff',
 'Unnamed: 0',
 'avg_comment_time',
 'max_comment_time',
 'std_comment_time',
 'comment_counts',
 'unique_speakers',
 'total_second_person_pronouns',
 'max_comment_length',
 'total_ats',
 'total_quotes',
 'median_words_in_comment',
 'std_dev_words_in_comment',
 'max_words_in_comment',
 'max_comments_by_one_speaker',
 'median_comments_by_one_speaker',
 'anger',
 'annoyance',
 'approval',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'pride',
 'sadness',
 'surprise',
 'has_neg_comment_sentcr',
 'non_neg_comment_ratio_sentcr',
 'neg_comment_ratio_sentcr',
 'sentiment_transition_ra

## Remove Used Conversations

In [62]:
# df_merged = df_merged[~df_merged['issue_url'].isin(new_labeled_urls)]
# df_merged

### Removing Issues for which LLama Could not Produce Label

In [63]:
df_merged = df_merged[~df_merged['issue_url'].isin(invalid_issue_urls)]

# Use Same Features as the Model

In [ ]:
# github-toxic/verifier-input-feature-generator/analysis/verifier_randomforest_models/feature_cols.pkl
feature_columns_df =  pd.read_csv(file_name)
feature_columns = feature_columns_df['features'].to_list()
feature_columns

['toxicity_score_llama',
 'toxicity_score_qwen',
 'is_toxic_llm_pred_llama',
 'is_toxic_llm_pred_qwen',
 'avg_toxicity_score',
 'toxicity_score_diff',
 'exp_first_token_logit',
 'unique_speakers',
 'symbol_to_word_ratio_#_TD',
 'non_toxic_prob_conv',
 'negative_word_count',
 'exp_first_token_entropy',
 'non_toxic_prob_exp',
 'total_non_neg_comment_textblob',
 'total_transitions_vader',
 'comment_counts',
 'sentiment_transition_ratio_vader',
 'max_words_in_comment',
 'max_comments_by_one_speaker',
 'toxic_prob_conv',
 'std_dev_words_in_comment',
 'n_tokens_TD',
 'total_transitions_textblob',
 'n_stop_words_TD',
 'toxic_prob_exp',
 'toxic_prob_conv_exp']

In [65]:
len(feature_columns)

26

In [66]:
df_merged = df_merged[['issue_url']+feature_columns]

In [67]:
# df_merged.to_csv("non_labelled_all_features.csv", index = False)

In [68]:
df_merged.columns

Index(['issue_url', 'toxicity_score_llama', 'toxicity_score_qwen',
       'is_toxic_llm_pred_llama', 'is_toxic_llm_pred_qwen',
       'avg_toxicity_score', 'toxicity_score_diff', 'exp_first_token_logit',
       'unique_speakers', 'symbol_to_word_ratio_#_TD', 'non_toxic_prob_conv',
       'negative_word_count', 'exp_first_token_entropy', 'non_toxic_prob_exp',
       'total_non_neg_comment_textblob', 'total_transitions_vader',
       'comment_counts', 'sentiment_transition_ratio_vader',
       'max_words_in_comment', 'max_comments_by_one_speaker',
       'toxic_prob_conv', 'std_dev_words_in_comment', 'n_tokens_TD',
       'total_transitions_textblob', 'n_stop_words_TD', 'toxic_prob_exp',
       'toxic_prob_conv_exp'],
      dtype='object')

In [69]:
# Assuming `series` is your Pandas Series
indices_with_none1 = df_merged['is_toxic_llm_pred_llama'][df_merged['is_toxic_llm_pred_llama'].isna()].index.tolist()
indices_with_none2 = df_merged['is_toxic_llm_pred_qwen'][df_merged['is_toxic_llm_pred_qwen'].isna()].index.tolist()

# Display the result
indices_with_none = indices_with_none1+indices_with_none2
indices_with_none

[]

In [70]:
df_merged = df_merged.drop(index=indices_with_none).reset_index(drop=True)
df_merged

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,sentiment_transition_ratio_vader,max_words_in_comment,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.1,False,False,0.150,0.10,1.353376,2,0.000100,...,0.000000,30,1,0.483464,14.142136,43,0.0,23.0,0.802078,0.651795
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,0.6,True,True,0.750,0.30,1.777933,6,0.006211,...,0.250000,107,2,0.574902,37.159646,336,3.0,176.0,0.860117,0.825229
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.0,False,False,0.100,0.20,2.790311,4,0.000100,...,0.500000,42,1,0.655699,14.545904,141,0.0,23.0,0.796883,0.783657
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,0.0,False,False,0.000,0.00,1.836710,2,0.000100,...,0.000000,95,1,0.679470,57.275649,139,0.0,49.0,0.572832,0.673987
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.0,False,False,0.025,0.05,0.491421,2,0.000100,...,0.000000,140,1,0.646438,96.166522,181,0.0,49.0,0.771985,0.773767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.0,False,False,0.100,0.20,2.751625,2,0.099729,...,0.428571,581,4,0.706250,196.462040,1589,3.0,512.0,0.842088,0.713442
105488,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,False,False,0.025,0.05,1.731196,1,0.127715,...,0.666667,981,3,0.683447,561.184462,1260,0.0,325.0,0.817401,0.785814
105489,https://api.github.com/repos/ytdl-org/youtube-...,0.10,0.0,False,False,0.050,0.10,1.556329,2,0.011749,...,0.500000,373,3,0.638400,131.067540,850,1.0,196.0,0.827594,0.728549
105490,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,False,False,0.025,0.05,1.948191,2,0.294466,...,0.500000,437,1,0.654645,280.721392,547,1.0,196.0,0.823427,0.724942


In [71]:
print(len(df_merged.columns))
df_merged.columns

27


Index(['issue_url', 'toxicity_score_llama', 'toxicity_score_qwen',
       'is_toxic_llm_pred_llama', 'is_toxic_llm_pred_qwen',
       'avg_toxicity_score', 'toxicity_score_diff', 'exp_first_token_logit',
       'unique_speakers', 'symbol_to_word_ratio_#_TD', 'non_toxic_prob_conv',
       'negative_word_count', 'exp_first_token_entropy', 'non_toxic_prob_exp',
       'total_non_neg_comment_textblob', 'total_transitions_vader',
       'comment_counts', 'sentiment_transition_ratio_vader',
       'max_words_in_comment', 'max_comments_by_one_speaker',
       'toxic_prob_conv', 'std_dev_words_in_comment', 'n_tokens_TD',
       'total_transitions_textblob', 'n_stop_words_TD', 'toxic_prob_exp',
       'toxic_prob_conv_exp'],
      dtype='object')

## Removing NAN

In [72]:
df_merged['is_toxic_llm_pred_llama'] = df_merged['is_toxic_llm_pred_llama'].astype(int)
df_merged['is_toxic_llm_pred_qwen'] = df_merged['is_toxic_llm_pred_qwen'].astype(int)


In [73]:
print("LLM predicted toxic samples: ", sum(df_merged['is_toxic_llm_pred_llama']))

LLM predicted toxic samples:  2533


In [74]:
# removing rows where llm couldnot predict properly
df_merged = df_merged[df_merged['is_toxic_llm_pred_llama'].isin([0,1])]
df_merged

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,sentiment_transition_ratio_vader,max_words_in_comment,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.1,0,0,0.150,0.10,1.353376,2,0.000100,...,0.000000,30,1,0.483464,14.142136,43,0.0,23.0,0.802078,0.651795
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,0.6,1,1,0.750,0.30,1.777933,6,0.006211,...,0.250000,107,2,0.574902,37.159646,336,3.0,176.0,0.860117,0.825229
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.0,0,0,0.100,0.20,2.790311,4,0.000100,...,0.500000,42,1,0.655699,14.545904,141,0.0,23.0,0.796883,0.783657
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,0.0,0,0,0.000,0.00,1.836710,2,0.000100,...,0.000000,95,1,0.679470,57.275649,139,0.0,49.0,0.572832,0.673987
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.0,0,0,0.025,0.05,0.491421,2,0.000100,...,0.000000,140,1,0.646438,96.166522,181,0.0,49.0,0.771985,0.773767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.0,0,0,0.100,0.20,2.751625,2,0.099729,...,0.428571,581,4,0.706250,196.462040,1589,3.0,512.0,0.842088,0.713442
105488,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.731196,1,0.127715,...,0.666667,981,3,0.683447,561.184462,1260,0.0,325.0,0.817401,0.785814
105489,https://api.github.com/repos/ytdl-org/youtube-...,0.10,0.0,0,0,0.050,0.10,1.556329,2,0.011749,...,0.500000,373,3,0.638400,131.067540,850,1.0,196.0,0.827594,0.728549
105490,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.948191,2,0.294466,...,0.500000,437,1,0.654645,280.721392,547,1.0,196.0,0.823427,0.724942


In [75]:
nan_columns = df_merged.columns[df_merged.isna().any()].tolist()

print(nan_columns)

[]


In [76]:
# removing rows with nan values
df_merged=df_merged.dropna(subset=nan_columns)
df_merged

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,sentiment_transition_ratio_vader,max_words_in_comment,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.1,0,0,0.150,0.10,1.353376,2,0.000100,...,0.000000,30,1,0.483464,14.142136,43,0.0,23.0,0.802078,0.651795
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,0.6,1,1,0.750,0.30,1.777933,6,0.006211,...,0.250000,107,2,0.574902,37.159646,336,3.0,176.0,0.860117,0.825229
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.0,0,0,0.100,0.20,2.790311,4,0.000100,...,0.500000,42,1,0.655699,14.545904,141,0.0,23.0,0.796883,0.783657
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,0.0,0,0,0.000,0.00,1.836710,2,0.000100,...,0.000000,95,1,0.679470,57.275649,139,0.0,49.0,0.572832,0.673987
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.0,0,0,0.025,0.05,0.491421,2,0.000100,...,0.000000,140,1,0.646438,96.166522,181,0.0,49.0,0.771985,0.773767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.0,0,0,0.100,0.20,2.751625,2,0.099729,...,0.428571,581,4,0.706250,196.462040,1589,3.0,512.0,0.842088,0.713442
105488,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.731196,1,0.127715,...,0.666667,981,3,0.683447,561.184462,1260,0.0,325.0,0.817401,0.785814
105489,https://api.github.com/repos/ytdl-org/youtube-...,0.10,0.0,0,0,0.050,0.10,1.556329,2,0.011749,...,0.500000,373,3,0.638400,131.067540,850,1.0,196.0,0.827594,0.728549
105490,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.948191,2,0.294466,...,0.500000,437,1,0.654645,280.721392,547,1.0,196.0,0.823427,0.724942


In [77]:
string_columns = df_merged.select_dtypes(include=['object', 'string']).columns

print("String columns:", list(string_columns))

String columns: ['issue_url']


## Feature Standardization

In [79]:
# replace infinite values with nan
df_merged[feature_columns] = df_merged[feature_columns].replace([np.inf, -np.inf], np.nan)


In [80]:
nan_columns = df_merged.columns[df_merged.isna().any()].tolist()

print(nan_columns)

[]


In [81]:
# Option 1: Drop rows with inf/nan values
df_merged = df_merged.dropna(subset=feature_columns)
df_merged

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,sentiment_transition_ratio_vader,max_words_in_comment,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.1,0,0,0.150,0.10,1.353376,2,0.000100,...,0.000000,30,1,0.483464,14.142136,43,0.0,23.0,0.802078,0.651795
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,0.6,1,1,0.750,0.30,1.777933,6,0.006211,...,0.250000,107,2,0.574902,37.159646,336,3.0,176.0,0.860117,0.825229
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.0,0,0,0.100,0.20,2.790311,4,0.000100,...,0.500000,42,1,0.655699,14.545904,141,0.0,23.0,0.796883,0.783657
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,0.0,0,0,0.000,0.00,1.836710,2,0.000100,...,0.000000,95,1,0.679470,57.275649,139,0.0,49.0,0.572832,0.673987
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.0,0,0,0.025,0.05,0.491421,2,0.000100,...,0.000000,140,1,0.646438,96.166522,181,0.0,49.0,0.771985,0.773767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.0,0,0,0.100,0.20,2.751625,2,0.099729,...,0.428571,581,4,0.706250,196.462040,1589,3.0,512.0,0.842088,0.713442
105488,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.731196,1,0.127715,...,0.666667,981,3,0.683447,561.184462,1260,0.0,325.0,0.817401,0.785814
105489,https://api.github.com/repos/ytdl-org/youtube-...,0.10,0.0,0,0,0.050,0.10,1.556329,2,0.011749,...,0.500000,373,3,0.638400,131.067540,850,1.0,196.0,0.827594,0.728549
105490,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.948191,2,0.294466,...,0.500000,437,1,0.654645,280.721392,547,1.0,196.0,0.823427,0.724942


In [82]:
string_columns = df_merged.select_dtypes(include=['object', 'string']).columns

print("String columns:", list(string_columns))

String columns: ['issue_url']


### Ramdomly Sample Conversations

In [84]:
st_scaler = joblib.load('./verifier_randomforest_models/standardScaler_70_30_split_explainable_features_reduced_with_qwen_final.pkl')

df_standardized = pd.DataFrame(st_scaler.fit_transform(df_merged[feature_columns]), columns=feature_columns)
df_standardized = df_standardized.reset_index(drop=True)
df_standardized['issue_url']=df_merged['issue_url'].reset_index(drop=True)
df_standardized

,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,non_toxic_prob_conv,...,max_words_in_comment,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp,issue_url
0,0.724862,1.069333,-0.156850,-0.051864,1.025909,0.030518,-0.301168,-0.318799,-0.652489,2.578619,...,-0.441497,-0.722878,-2.563928,-0.427464,-0.484952,-0.644755,-0.575217,0.277038,-1.258054,https://api.github.com/repos/2noise/ChatTTS/is...
1,6.556859,8.811877,6.375505,19.281163,8.914532,1.904728,0.201633,2.529873,-0.554267,1.391564,...,-0.317658,-0.162270,-1.328327,-0.351282,-0.244352,1.184169,0.081829,1.057392,1.262494,https://api.github.com/repos/2noise/ChatTTS/is...
2,0.724862,-0.479176,-0.156850,-0.051864,0.368524,0.967623,1.400589,1.105537,-0.652489,0.240334,...,-0.422197,-0.722878,-0.236524,-0.426128,-0.404478,-0.644755,-0.575217,0.207195,0.658324,https://api.github.com/repos/2noise/ChatTTS/is...
3,-0.941423,-0.479176,-0.156850,-0.051864,-0.946247,-0.906587,0.271244,-0.318799,-0.652489,-0.019956,...,-0.336958,-0.722878,0.084701,-0.284703,-0.406121,-0.644755,-0.463562,-2.805232,-0.935535,https://api.github.com/repos/2noise/ChatTTS/is...
4,-0.524852,-0.479176,-0.156850,-0.051864,-0.617554,-0.438035,-1.321977,-0.318799,-0.652489,0.409059,...,-0.264585,-0.722878,-0.361660,-0.155984,-0.371632,-0.644755,-0.463562,-0.127563,0.514592,https://api.github.com/repos/2noise/ChatTTS/is...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,0.724862,-0.479176,-0.156850,-0.051864,0.368524,0.967623,1.354773,-0.318799,0.948797,-0.414815,...,0.444672,0.958947,0.446571,0.175970,0.784559,1.184169,1.524756,0.814988,-0.362128,https://api.github.com/repos/ytdl-org/youtube-...
105488,-0.524852,-0.479176,-0.156850,-0.051864,-0.617554,-0.438035,0.146284,-1.030967,1.398610,-0.137152,...,1.087988,0.398339,0.138431,1.383113,0.514398,-0.644755,0.721699,0.483064,0.689665,https://api.github.com/repos/ytdl-org/youtube-...
105489,-0.108280,-0.479176,-0.156850,-0.051864,-0.288861,0.030518,-0.060810,-0.318799,-0.465255,0.492061,...,0.110147,0.398339,-0.470276,-0.040470,0.177723,-0.035113,0.167718,0.620116,-0.142570,https://api.github.com/repos/ytdl-org/youtube-...
105490,-0.524852,-0.479176,-0.156850,-0.051864,-0.617554,-0.438035,0.403270,-0.318799,4.078731,0.224328,...,0.213078,-0.722878,-0.250758,0.454848,-0.071088,-0.035113,0.167718,0.564086,-0.195001,https://api.github.com/repos/ytdl-org/youtube-...


In [85]:
nan_columns = df_standardized.columns[df_standardized.isna().any()].tolist()

print(nan_columns)

[]


In [86]:
print(df_standardized[nan_columns].isna().sum())  # Count NaNs in specified columns

Series([], dtype: float64)


In [87]:
if 'per_word_perplexity_TD' in df_merged.columns: 
    print(df_merged['per_word_perplexity_TD'].std() )# as std is infinite we need to drop it

In [88]:
if 'symbol_to_word_ratio_#_TD' in df_merged.columns: 
    print("min value:", df_merged['symbol_to_word_ratio_#_TD'].min() )
    print("max value:", df_merged['symbol_to_word_ratio_#_TD'].max() )
    print("median:",df_merged['symbol_to_word_ratio_#_TD'].median() )
    print("std:", df_merged['symbol_to_word_ratio_#_TD'].std() )# as std is infinite we need to drop it 

min value: 0.0001
max value: 3.242914979757085
median: 0.0214592274678111
std: 0.062218152724046766


In [89]:
df_standardized=df_standardized.drop(columns=nan_columns)

df_standardized

,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,non_toxic_prob_conv,...,max_words_in_comment,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp,issue_url
0,0.724862,1.069333,-0.156850,-0.051864,1.025909,0.030518,-0.301168,-0.318799,-0.652489,2.578619,...,-0.441497,-0.722878,-2.563928,-0.427464,-0.484952,-0.644755,-0.575217,0.277038,-1.258054,https://api.github.com/repos/2noise/ChatTTS/is...
1,6.556859,8.811877,6.375505,19.281163,8.914532,1.904728,0.201633,2.529873,-0.554267,1.391564,...,-0.317658,-0.162270,-1.328327,-0.351282,-0.244352,1.184169,0.081829,1.057392,1.262494,https://api.github.com/repos/2noise/ChatTTS/is...
2,0.724862,-0.479176,-0.156850,-0.051864,0.368524,0.967623,1.400589,1.105537,-0.652489,0.240334,...,-0.422197,-0.722878,-0.236524,-0.426128,-0.404478,-0.644755,-0.575217,0.207195,0.658324,https://api.github.com/repos/2noise/ChatTTS/is...
3,-0.941423,-0.479176,-0.156850,-0.051864,-0.946247,-0.906587,0.271244,-0.318799,-0.652489,-0.019956,...,-0.336958,-0.722878,0.084701,-0.284703,-0.406121,-0.644755,-0.463562,-2.805232,-0.935535,https://api.github.com/repos/2noise/ChatTTS/is...
4,-0.524852,-0.479176,-0.156850,-0.051864,-0.617554,-0.438035,-1.321977,-0.318799,-0.652489,0.409059,...,-0.264585,-0.722878,-0.361660,-0.155984,-0.371632,-0.644755,-0.463562,-0.127563,0.514592,https://api.github.com/repos/2noise/ChatTTS/is...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,0.724862,-0.479176,-0.156850,-0.051864,0.368524,0.967623,1.354773,-0.318799,0.948797,-0.414815,...,0.444672,0.958947,0.446571,0.175970,0.784559,1.184169,1.524756,0.814988,-0.362128,https://api.github.com/repos/ytdl-org/youtube-...
105488,-0.524852,-0.479176,-0.156850,-0.051864,-0.617554,-0.438035,0.146284,-1.030967,1.398610,-0.137152,...,1.087988,0.398339,0.138431,1.383113,0.514398,-0.644755,0.721699,0.483064,0.689665,https://api.github.com/repos/ytdl-org/youtube-...
105489,-0.108280,-0.479176,-0.156850,-0.051864,-0.288861,0.030518,-0.060810,-0.318799,-0.465255,0.492061,...,0.110147,0.398339,-0.470276,-0.040470,0.177723,-0.035113,0.167718,0.620116,-0.142570,https://api.github.com/repos/ytdl-org/youtube-...
105490,-0.524852,-0.479176,-0.156850,-0.051864,-0.617554,-0.438035,0.403270,-0.318799,4.078731,0.224328,...,0.213078,-0.722878,-0.250758,0.454848,-0.071088,-0.035113,0.167718,0.564086,-0.195001,https://api.github.com/repos/ytdl-org/youtube-...


## Load Models

In [ ]:
file_path = '/path/to/input/files' % replace with real path

loaded_model = joblib.load(file_path)


In [91]:
def predict_with_model(model, X, return_probabilities=True):
    """
    Make predictions using a loaded model
    
    Parameters:
    model: loaded sklearn model
    X (pandas.DataFrame): Features to predict on
    return_probabilities (bool): If True, return probability scores along with predictions
    
    Returns:
    tuple or array: If return_probabilities is True, returns (predictions, probabilities),
                   otherwise returns just predictions
    """
    predictions = model.predict(X)
    if return_probabilities:
        probabilities = model.predict_proba(X)
        return predictions, probabilities[:, 1]
    return predictions


In [92]:
# Make predictions using ensemble of all models
_, probabilities = predict_with_model(loaded_model, df_standardized[feature_columns])
predictions = probabilities>=0.4
df_merged['is_correct_verifier_pred_probability'] = probabilities
df_merged['is_correct_verifier_pred_label'] = predictions


In [93]:
for p in probabilities:
    print(p)

0.55
0.97
0.61
0.91
0.88
0.55
0.74
0.61
0.36
0.85
0.88
0.68
0.88
0.46
0.69
0.88
0.68
0.89
0.87
0.93
0.63
0.88
0.91
0.59
0.64
0.87
0.81
0.89
0.88
0.92
0.83
0.88
0.88
0.7
0.91
0.91
0.91
0.89
0.58
0.68
0.86
0.61
0.89
0.9
0.54
0.84
0.73
0.47
0.93
0.86
0.34
0.6
0.93
0.9
0.63
0.65
0.91
0.92
0.67
0.89
0.5
0.68
0.69
0.61
0.63
0.63
0.62
0.9
0.63
0.86
0.53
0.64
0.87
0.88
0.87
0.49
0.83
0.87
0.57
0.64
0.88
0.74
0.91
0.38
0.65
0.8
0.52
0.91
0.62
0.91
0.69
0.78
0.47
0.52
0.65
0.65
0.62
0.64
0.44
0.74
0.7
0.89
0.56
0.62
0.54
0.56
0.63
0.63
0.73
0.58
0.89
0.36
0.47
0.26
0.89
0.67
0.84
0.89
0.83
0.87
0.8
0.58
0.25
0.43
0.68
0.58
0.7
0.57
0.69
0.92
0.86
0.65
0.63
0.47
0.91
0.47
0.68
0.73
0.92
0.59
0.39
0.64
0.92
0.89
0.92
0.4
0.79
0.74
0.92
0.91
0.48
0.93
0.56
0.7
0.57
0.41
0.89
0.88
0.64
0.47
0.5
0.39
0.42
0.6
0.42
0.91
0.86
0.68
0.91
0.67
0.53
0.92
0.65
0.86
0.89
0.64
0.89
0.91
0.88
0.6
0.88
0.81
0.87
0.39
0.64
0.71
0.89
0.9
0.85
0.82
0.92
0.71
0.36
0.67
0.58
0.88
0.93
0.87
0.67
0.41
0.64
0.9
0.83
0.

In [94]:
incorrect_df = df_merged[df_merged['is_correct_verifier_pred_label']==False]
total_incorrect = incorrect_df.shape[0]

print(f'Total incorrect prediction by verifier is {total_incorrect} out of {df_merged.shape[0]}')

Total incorrect prediction by verifier is 10248 out of 105492


In [95]:
print(f"Total llama predicted toxic conversation is {df_merged[df_merged['is_toxic_llm_pred_llama']==True].shape[0]} out of {df_merged.shape[0]}")

Total llama predicted toxic conversation is 2533 out of 105492


In [96]:
correct_df = df_merged[df_merged['is_correct_verifier_pred_label']==True]

correct_df

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp,is_correct_verifier_pred_probability,is_correct_verifier_pred_label
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.1,0,0,0.150,0.10,1.353376,2,0.000100,...,1,0.483464,14.142136,43,0.0,23.0,0.802078,0.651795,0.55,True
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,0.6,1,1,0.750,0.30,1.777933,6,0.006211,...,2,0.574902,37.159646,336,3.0,176.0,0.860117,0.825229,0.97,True
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.0,0,0,0.100,0.20,2.790311,4,0.000100,...,1,0.655699,14.545904,141,0.0,23.0,0.796883,0.783657,0.61,True
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,0.0,0,0,0.000,0.00,1.836710,2,0.000100,...,1,0.679470,57.275649,139,0.0,49.0,0.572832,0.673987,0.91,True
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.0,0,0,0.025,0.05,0.491421,2,0.000100,...,1,0.646438,96.166522,181,0.0,49.0,0.771985,0.773767,0.88,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.0,0,0,0.100,0.20,2.751625,2,0.099729,...,4,0.706250,196.462040,1589,3.0,512.0,0.842088,0.713442,0.59,True
105488,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.731196,1,0.127715,...,3,0.683447,561.184462,1260,0.0,325.0,0.817401,0.785814,0.91,True
105489,https://api.github.com/repos/ytdl-org/youtube-...,0.10,0.0,0,0,0.050,0.10,1.556329,2,0.011749,...,3,0.638400,131.067540,850,1.0,196.0,0.827594,0.728549,0.77,True
105490,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.948191,2,0.294466,...,1,0.654645,280.721392,547,1.0,196.0,0.823427,0.724942,0.91,True


In [97]:
correct_toxic_df = correct_df[correct_df['is_toxic_llm_pred_llama']==True]
print(f"{correct_toxic_df.shape[0]} LLM predicted toxic are  in {correct_df.shape[0]} correct predictions")

2533 LLM predicted toxic are  in 95244 correct predictions


In [98]:
incorrect_df = df_merged[df_merged['is_correct_verifier_pred_label']==False]

incorrect_df = incorrect_df.merge(df_conversation, on='issue_url', how='left')
incorrect_df

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,conversation,repo,id,comments_url,created_at,user,author_association,locked,comments_count,title
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.10,0,0,0.075,0.05,1.246644,4,0.000100,...,Rick-ustc: WARNING:ChatTTS.utils.gpu_utils:No ...,2noise/ChatTTS,2330106497,https://api.github.com/repos/2noise/ChatTTS/is...,2024-06-03 03:51:33,Rick-ustc,NONE,0,3,mac m1报错呀，尝试改成mps，好像也是报这个错
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.10,0,0,0.150,0.10,2.593040,2,0.054795,...,spxneo: ```py\r\nimport ChatTTS\r\nimport torc...,2noise/ChatTTS,2441554442,https://api.github.com/repos/2noise/ChatTTS/is...,2024-08-01 06:02:11,spxneo,NONE,0,2,refine codes are read literally
2,https://api.github.com/repos/All-Hands-AI/Open...,0.20,0.00,0,0,0.100,0.20,1.596572,4,0.004085,...,hussner: <!-- You MUST fill out this template....,All-Hands-AI/OpenHands,2236485659,https://api.github.com/repos/All-Hands-AI/Open...,2024-04-10 21:49:49,hussner,NONE,0,14,azureopenai - embedding operation does not wor...
3,https://api.github.com/repos/All-Hands-AI/Open...,0.20,0.00,0,0,0.100,0.20,1.678946,5,0.006321,...,katmai: <!-- You MUST fill out this template. ...,All-Hands-AI/OpenHands,2239992726,https://api.github.com/repos/All-Hands-AI/Open...,2024-04-12 11:59:48,katmai,NONE,0,8,ERROR:root:<class 'pexpect.pxssh.ExceptionPxss...
4,https://api.github.com/repos/All-Hands-AI/Open...,0.05,0.00,0,0,0.025,0.05,0.800009,7,0.006806,...,2868151647: <!-- You MUST fill out this templa...,All-Hands-AI/OpenHands,2240185928,https://api.github.com/repos/All-Hands-AI/Open...,2024-04-12 13:41:29,2868151647,NONE,0,16,"Use llama2 locally, keep requesting '/chat/com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10243,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.05,0,0,0.125,0.15,1.618398,3,0.004932,...,DmytroUsenko: - [X] I've verified that I'm run...,ytdl-org/youtube-dl,2447081495,https://api.github.com/repos/ytdl-org/youtube-...,2024-08-04 10:58:19,DmytroUsenko,NONE,0,18,[Youtube] Get ERROR: giving up after 0 fragmen...
10244,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.10,0,0,0.075,0.05,1.253408,2,0.045833,...,"wjandrea: Python 2 is now unsupported, so use ...",ytdl-org/youtube-dl,2448975967,https://api.github.com/repos/ytdl-org/youtube-...,2024-08-05 16:29:00,wjandrea,NONE,0,2,README.md: replace outdated Python doc link
10245,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.10,0,0,0.075,0.05,1.070461,3,0.124270,...,alfonsomoratalla: <!--\r\n\r\n################...,ytdl-org/youtube-dl,2450482586,https://api.github.com/repos/ytdl-org/youtube-...,2024-08-06 10:06:26,alfonsomoratalla,NONE,0,2,Youtube not working
10246,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.10,0,0,0.075,0.05,3.203459,3,0.246256,...,finid: <!--\r\n\r\n###########################...,ytdl-org/youtube-dl,2454266383,https://api.github.com/repos/ytdl-org/youtube-...,2024-08-07 20:14:54,finid,NONE,0,4,Suntax error in Linux


In [99]:
incorrect_df.to_csv('./output/large_dataset_validated_incorrect_conversations_final.csv', index = False)

In [100]:
correct_df = df_merged[df_merged['is_correct_verifier_pred_label']==True]
correct_df

,issue_url,toxicity_score_llama,toxicity_score_qwen,is_toxic_llm_pred_llama,is_toxic_llm_pred_qwen,avg_toxicity_score,toxicity_score_diff,exp_first_token_logit,unique_speakers,symbol_to_word_ratio_#_TD,...,max_comments_by_one_speaker,toxic_prob_conv,std_dev_words_in_comment,n_tokens_TD,total_transitions_textblob,n_stop_words_TD,toxic_prob_exp,toxic_prob_conv_exp,is_correct_verifier_pred_probability,is_correct_verifier_pred_label
0,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.1,0,0,0.150,0.10,1.353376,2,0.000100,...,1,0.483464,14.142136,43,0.0,23.0,0.802078,0.651795,0.55,True
1,https://api.github.com/repos/2noise/ChatTTS/is...,0.90,0.6,1,1,0.750,0.30,1.777933,6,0.006211,...,2,0.574902,37.159646,336,3.0,176.0,0.860117,0.825229,0.97,True
2,https://api.github.com/repos/2noise/ChatTTS/is...,0.20,0.0,0,0,0.100,0.20,2.790311,4,0.000100,...,1,0.655699,14.545904,141,0.0,23.0,0.796883,0.783657,0.61,True
3,https://api.github.com/repos/2noise/ChatTTS/is...,0.00,0.0,0,0,0.000,0.00,1.836710,2,0.000100,...,1,0.679470,57.275649,139,0.0,49.0,0.572832,0.673987,0.91,True
4,https://api.github.com/repos/2noise/ChatTTS/is...,0.05,0.0,0,0,0.025,0.05,0.491421,2,0.000100,...,1,0.646438,96.166522,181,0.0,49.0,0.771985,0.773767,0.88,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105487,https://api.github.com/repos/ytdl-org/youtube-...,0.20,0.0,0,0,0.100,0.20,2.751625,2,0.099729,...,4,0.706250,196.462040,1589,3.0,512.0,0.842088,0.713442,0.59,True
105488,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.731196,1,0.127715,...,3,0.683447,561.184462,1260,0.0,325.0,0.817401,0.785814,0.91,True
105489,https://api.github.com/repos/ytdl-org/youtube-...,0.10,0.0,0,0,0.050,0.10,1.556329,2,0.011749,...,3,0.638400,131.067540,850,1.0,196.0,0.827594,0.728549,0.77,True
105490,https://api.github.com/repos/ytdl-org/youtube-...,0.05,0.0,0,0,0.025,0.05,1.948191,2,0.294466,...,1,0.654645,280.721392,547,1.0,196.0,0.823427,0.724942,0.91,True


In [101]:
correct_df.to_csv('./output/large_dataset_validated_correct_conversations_final.csv', index = False)